# Import thư viện cần thiết

In [27]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

Điều chính kích thước và độ phóng chung cho các biểu đồ trong bài tập này.

In [28]:
plt.rcParams['figure.figsize'] = [12, 7]
plt.rcParams['figure.dpi'] = 100

Đọc dữ liệu

In [29]:
train = pd.read_csv('../data/train_preprocessed.csv')
train.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [30]:
test = pd.read_csv('../data/test_preprocessed.csv')
test.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,1,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,121.00,324


Ta sẽ tổng hợp 2 bộ dữ liệu lại để có cái nhìn tổng quan về dữ liệu này.

In [31]:
ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4.0
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4.0
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4.0
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4.0
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4.0


# Xây dựng mô hình

### Tạo biến ordinal từ dữ liệu đã được one-hot encode 

In [32]:
def get_numeric(data, status_name):
    status_cols = [s for s in data.columns.tolist() if status_name in s]
    status_df = data[status_cols]
    status_df.columns = list(range(status_df.shape[1]))
    status_numeric = status_df.idxmax(1)
    status_numeric.name = status_name
    data = pd.concat([data, status_numeric], axis=1)
    return data

In [33]:
status_name_list = ['epared', 'etecho', 'eviv', 'instlevel']
for status_name in status_name_list:
    all_data = get_numeric(all_data, status_name)

AttributeError: 'int' object has no attribute 'max'

In [ ]:
def get_tipovivi_value(row):
    if row['tipovivi1'] == 1:
        return 4
    elif row['tipovivi2'] == 1:
        return 3
    elif row['tipovivi3'] == 1:
        return 2
    elif row['tipovivi4'] == 1:
        return 1
    elif row['tipovivi5'] == 1:
        return 0
    else:
        return None  

all_data['tipovivi'] =  all_data.apply(get_tipovivi_value, axis=1)

def get_abastagua_value(row):
    if row['abastaguadentro'] == 1:
        return 2
    elif row['abastaguafuera'] == 1:
        return 1
    elif row['abastaguano'] == 1:
        return 0
    else:
        return None 

all_data['abastagua'] =  all_data.apply(get_abastagua_value, axis=1)

### Xoá các cột không cần thiết

1. Cột `v18q` cho biết hộ gia đình đó có máy tính bảng hay không và `v18q1` chỉ số lượng máy tính bảng của hộ gia đình đó. Do đó việc biết hộ gia đình có máy tính bảng hay không là quan trọng hơn nên ta sẽ xóa cột `v18q1`.

In [ ]:
selected_columns = all_data.loc[:, ['v18q', 'v18q1']]
selected_columns

,v18q,v18q1
0,0,0.0
1,1,1.0
2,0,0.0
3,1,1.0
4,1,1.0
...,...,...
33408,0,0.0
33409,0,0.0
33410,0,0.0
33411,0,0.0


2. Ta thấy các cột sao giống nhau về ý nghĩa và giá trị
- `tamhog` -  size of the household
- `tamviv` - number of persons living in the household
- `hhsize` - household size
- `r4t3` - total persons in the household
- `hogar_total` -  total individuals in the household

In [ ]:
selected_columns = all_data.loc[:, ['tamviv', 'hhsize', 'tamhog', 'hogar_total', 'r4t3']]
selected_columns

,tamviv,hhsize,tamhog,hogar_total,r4t3
0,1,1,1,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,4,4,4,4,4
4,4,4,4,4,4
...,...,...,...,...,...
33408,6,6,6,6,6
33409,4,4,4,4,4
33410,4,4,4,4,4
33411,4,4,4,4,4


Ta sẽ xoá 4 thuộc tính  `tamhog`, `tamviv`, `hhsize`, `r4t3` và chừa lại `hogar_total` để giữ lại thông tin về số người trong gia đình.

3. `edjefe` và `edjefa` chỉ số năm học vấn của chủ hộ, tuy nhiên ta cũng có thuộc tính chỉ trình độ học vấn là `instlevel` nên ta sẽ xoá `edjefe` và `edjefa`.
4. `meaneduc` chỉ trung bình số năm học vấn của người lớn tuy nhiên ta thấy cột này không mang lại nhièu ý nghĩa so với thuộc tính trình độ học vấn nên ta sẽ xoá đi.
5. `female` va `male`. Ta sẽ xoá cột `female` 

In [ ]:
correlation = all_data['female'].corr(all_data['male'])
print("Correlation between female and male:", correlation)

Correlation between female and male: -0.9999999999999981


6. `overcrowding` chỉ số người trung bình trong các phòng ngủ, ta thấy cột này cũng không cần thiết

In [ ]:
selected_columns = all_data.loc[:, ['overcrowding','bedrooms','hhsize']]
selected_columns

,overcrowding,bedrooms,hhsize
0,1.000000,1,1
1,1.000000,1,1
2,0.500000,2,1
3,1.333333,3,4
4,1.333333,3,4
...,...,...,...
33408,6.000000,1,6
33409,2.000000,2,4
33410,2.000000,2,4
33411,2.000000,2,4


7. `mobilephone` cho biết hộ gia đình đó có điện thoại di động không trong khi `qmobilephone` lại cho biết số lượng điện thoại di động trong hộ gia đình đó, nên `mobilephone` sẽ trở nên không cần thiết

In [ ]:
selected_columns = all_data.loc[:, ['mobilephone','qmobilephone']]
selected_columns

,mobilephone,qmobilephone
0,1,1
1,1,1
2,0,0
3,1,3
4,1,3
...,...,...
33408,1,1
33409,1,2
33410,1,2
33411,1,2


8. `area1` và `area2`. Ta sẽ xoá đi `area2` vì nó chỉ là giá trị đảo ngược của `area1`.

In [ ]:
correlation = all_data['area1'].corr(all_data['area2'])
print("Correlation between area1 and area2:", correlation)

Correlation between area1 and area2: -1.0


9. Xét các thuộc tính sau:
- `r4h1` - Males younger than 12 years of age
- `r4h2` - Males 12 years of age and older
- `r4h3` - Total males in the household
- `r4m1` - Females younger than 12 years of age
- `r4m2` - Females 12 years of age and older
- `r4m3` - Total females in the household
- `r4t1` - persons younger than 12 years of age
- `r4t2` - persons 12 years of age and older
- `r4t3` - Total persons in the household

Số lượng tổng có thể suy ra từ số lượng nam và nữ, mặc khác ta cũng đã xoá đi thuộc tính `r4t3` ở trên nên ta cũng sẽ xoá luôn 2 thuộc tính `r4t1` và `r4t2`. 

10. Xoá `estadocivil1` (=1 if less than 10 years old)
11. `epared1~3`, `etecho1~3`, `eviv1~3`, `instlevel1~9`, `tipovivi1~5`, `abastaguadentro`, `abastaguafuera`, `abastaguano` 

In [ ]:
redundant_features = [  'v18q', 'tamviv', 'hhsize', 'tamhog', 'r4t3',
                        'abastaguadentro', 'abastaguafuera', 'abastaguano', 'estadocivil1', 'r4t1', 'r4t2', 'area2',
                        'mobilephone', 'overcrowding', 'female', 'meaneduc', 'edjefe', 'edjefa',
                      'epared1', 'epared2', 'epared3', 
                      'etecho1', 'etecho2', 'etecho3',
                      'eviv1', 'eviv2', 'eviv3', 
                      'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',
                      'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5' 
]
all_data.drop(columns=redundant_features, inplace=True)

### Trích lọc đặc trưng bằng thông số thống kê

In [ ]:
ind_bool = ['dis', 'male',
            'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 
            'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12', 'instlevel']

ind_ordered = ['escolari', 'age']

In [ ]:
range = lambda x: x.max() - x.min()
range.__name__ = 'range_'
ind_agg = all_data.groupby('idhogar')[ind_ordered + ind_bool].agg(['mean', 'max', 'min', 'sum', 'count', 'std', range])

new_cols = []
for col in ind_agg.columns.levels[0]:
    for stat in ind_agg.columns.levels[1]:
        new_cols.append(f'{col}-{stat}')

ind_agg.columns = new_cols
ind_agg.head()

,escolari-mean,escolari-max,escolari-min,escolari-sum,escolari-count,escolari-std,escolari-range_,age-mean,age-max,age-min,...,parentesco12-count,parentesco12-std,parentesco12-range_,instlevel-mean,instlevel-max,instlevel-min,instlevel-sum,instlevel-count,instlevel-std,instlevel-range_
idhogar,,,,,,,,,,,,,,,,,,,,,
000a08204,8.666667,14,0,26,3,7.571878,14,20.666667,30,4,...,3,0.0,0,4.666667,7,0,14,3,4.041452,7
000bce7c4,2.500000,5,0,5,2,3.535534,5,61.500000,63,60,...,2,0.0,0,0.500000,1,0,1,2,0.707107,1
001845fb0,10.250000,14,6,41,4,3.304038,8,35.500000,52,19,...,4,0.0,0,4.000000,7,2,16,4,2.160247,5
001ff74ca,8.000000,16,0,16,2,11.313708,16,19.000000,38,0,...,2,0.0,0,3.500000,7,0,7,2,4.949747,7
003123ec2,3.250000,7,0,13,4,3.774917,7,12.750000,24,1,...,4,0.0,0,1.250000,3,0,5,4,1.500000,3


In [ ]:
corr_matrix = ind_agg.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any((abs(upper[column]) > 0.95) & (abs(upper[column]) == 1))]
print(f'There are {len(to_drop)} correlated columns to remove')

all_data = all_data.merge(ind_agg, on = 'idhogar', how = 'left')
all_data.drop(columns=ind_bool+ind_ordered+to_drop, inplace=True)
print('Number of features after dropping the individual level features', all_data.shape[1])

There are 32 correlated columns to remove
Number of features after dropping the individual level features 214


### Tạo thêm cột feature mới

In [ ]:
def extract_features(all_data):
    # Monthly rent per room
    all_data['monthly_rent_per_room'] = all_data['v2a1'] / all_data['rooms']
    # Monthly rent per adult
    all_data['monthly_rent_per_adult'] = all_data['v2a1'] / all_data['hogar_adul']
    # No. of adults per room
    all_data['adults_per_room'] = all_data['hogar_adul'] / all_data['rooms']
    # No. of adults per bedroom
    all_data['adults_per_bedroom'] = all_data['hogar_adul'] / all_data['bedrooms']

for df in [train, test]:
    extract_features(df)

### Chuẩn bị dữ liệu

In [ ]:
train = all_data[:ntrain][:]
test = all_data[ntrain:][:]

Lấy dữ liệu dùng để huấn luyện model từ tập train.

In [ ]:
X = train.drop(columns=['Id', 'idhogar', 'Target'])
y = train.Target.to_numpy().astype('int') - 1

Lấy dữ liệu dùng để huấn luyện từ tập test.


In [ ]:

data_test_id = list(test.Id)
data_test = test.drop(columns=['Id', 'idhogar', 'Target'])

Chia dữ liệu huấn luyện thành tập train và test.


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

# Khởi tạo mô hình XGBoost
model_xgb = XGBClassifier(
    booster='gbtree',
    learning_rate=0.1,
    n_estimators=1200,
    max_depth=6,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softmax',
    num_class=4,
    reg_alpha=0,
    reg_lambda=1,
    random_state=42,
    verbosity=0
) 

# Huấn luyện mô hình
model_xgb.fit(X_train, y_train)

# Đánh giá mô hình
print(classification_report(y_test, model_xgb.predict(X_test), target_names=['class 1', 'class 2', 'class 3', 'class 4']))


              precision    recall  f1-score   support

     class 1       0.91      0.84      0.88       244
     class 2       0.88      0.88      0.88       472
     class 3       0.91      0.83      0.87       380
     class 4       0.95      0.98      0.97      1772

    accuracy                           0.93      2868
   macro avg       0.92      0.88      0.90      2868
weighted avg       0.93      0.93      0.93      2868



In [ ]:
print("F1 score macro: ", f1_score(y_test, model_xgb.predict(X_test), average = 'macro'))


F1 score macro:  0.8994441074846776


### Submit

In [ ]:
useless_xgb_cols = X_train.columns[model_xgb.feature_importances_ == 0].tolist()
model_xgb = model_xgb.fit(X_train.drop(columns = useless_xgb_cols), y_train)
xgb_submission = pd.DataFrame({'Id': data_test_id, 'Target': model_xgb.predict(data_test.drop(columns = useless_xgb_cols))+1})
xgb_submission.to_csv('submission.csv', index = False)